In [ ]:
import pandas as pd
stock_data=pd.read_csv('NFLX (1).csv', index_col='Date')
stock_data.head()

Open        High         Low       Close   Adj Close  \
Date                                                                     
2019-03-11  352.000000  358.980011  350.029999  358.859985  358.859985   
2019-03-12  359.369995  360.130005  353.799988  356.269989  356.269989   
2019-03-13  355.809998  362.480011  352.769989  361.209991  361.209991   
2019-03-14  360.500000  363.839996  358.100006  358.820007  358.820007   
2019-03-15  361.019989  364.000000  358.890015  361.459991  361.459991   

             Volume  
Date                 
2019-03-11  5387300  
2019-03-12  5164600  
2019-03-13  6444100  
2019-03-14  5271400  
2019-03-15  8444500

In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import datetime as dt
plt.figure(figsize=(15,10))
plt.gca().xaxis.set_major_formatter(mdates. DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=60))
x_dates = [dt.datetime.strptime(d, '%Y-%m-%d').date() for d in stock_data.index.values]

plt.plot(x_dates, stock_data['High'], label='High')
plt.plot(x_dates, stock_data['Low'], label='Low')
plt.xlabel('Time Scale')
plt.ylabel('Scaled USD')
plt. legend ()
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
from tensorflow.keras. models import Sequential 
from tensorflow. keras.layers import Dense 
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dropout 
from tensorflow. keras. layers import *
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_percentage_error 
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.metrics import mean_squared_error

In [ ]:
target_y = stock_data['Close']
X_feat = stock_data.iloc[:,0:3]

In [ ]:
X_feat

Open        High         Low
Date                                          
2019-03-11  352.000000  358.980011  350.029999
2019-03-12  359.369995  360.130005  353.799988
2019-03-13  355.809998  362.480011  352.769989
2019-03-14  360.500000  363.839996  358.100006
2019-03-15  361.019989  364.000000  358.890015
...                ...         ...         ...
2023-03-03  315.450012  317.489990  310.820007
2023-03-06  317.000000  323.299988  311.839996
2023-03-07  312.679993  314.299988  306.619995
2023-03-08  309.290009  311.829987  305.750000
2023-03-09  312.079987  312.510010  294.880005

[1008 rows x 3 columns]

In [ ]:
sc = StandardScaler()
X_ft = sc.fit_transform(X_feat. values)
X_ft = pd.DataFrame (columns=X_feat. columns, data=X_ft, index=X_feat.index)

In [ ]:
len(X_ft)

1008

In [ ]:
def lstm_split (data, n_steps) :
  X, y = [], []
  for i in range(len(data)-n_steps+1):
    X.append(data[i:i+n_steps,:-1]) 
    y.append(data[i+n_steps-1,-1])
  return np.array(X), np.array (y)  

In [ ]:
X1, y1 = lstm_split(X_ft.values, n_steps=2)
print(y1.shape)
train_split=0.8
split_idx = int(np.ceil(len(X1)*train_split))
date_index = X_ft.index

X_train, X_test = X1[:split_idx], X1[split_idx: ]
y_train, y_test = y1[:split_idx], y1[split_idx:]
x_train_date, X_test_date = date_index[:split_idx], date_index[split_idx: ]
print(X1.shape, X_train.shape, X_test.shape, y_test.shape)

(1007,)
(1007, 2, 2) (806, 2, 2) (201, 2, 2) (201,)


In [ ]:
lstm = Sequential()
lstm.add(LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu', return_sequences=False))
lstm.add(Dense(1))
lstm.compile(loss='mean_squared_error', optimizer='adam')
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                4480      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 4,513
Trainable params: 4,513
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history=lstm.fit(X_train, y_train,epochs=100, batch_size=4, verbose=2, shuffle=False)

Epoch 1/100
202/202 - 2s - loss: 0.2051 - 2s/epoch - 12ms/step
Epoch 2/100
202/202 - 1s - loss: 0.0106 - 557ms/epoch - 3ms/step
Epoch 3/100
202/202 - 1s - loss: 0.0077 - 521ms/epoch - 3ms/step
Epoch 4/100
202/202 - 1s - loss: 0.0068 - 550ms/epoch - 3ms/step
Epoch 5/100
202/202 - 1s - loss: 0.0055 - 516ms/epoch - 3ms/step
Epoch 6/100
202/202 - 1s - loss: 0.0052 - 596ms/epoch - 3ms/step
Epoch 7/100
202/202 - 1s - loss: 0.0047 - 512ms/epoch - 3ms/step
Epoch 8/100
202/202 - 1s - loss: 0.0044 - 515ms/epoch - 3ms/step
Epoch 9/100
202/202 - 1s - loss: 0.0042 - 706ms/epoch - 3ms/step
Epoch 10/100
202/202 - 1s - loss: 0.0040 - 751ms/epoch - 4ms/step
Epoch 11/100
202/202 - 1s - loss: 0.0039 - 886ms/epoch - 4ms/step
Epoch 12/100
202/202 - 1s - loss: 0.0037 - 732ms/epoch - 4ms/step
Epoch 13/100
202/202 - 1s - loss: 0.0036 - 616ms/epoch - 3ms/step
Epoch 14/100
202/202 - 1s - loss: 0.0035 - 869ms/epoch - 4ms/step
Epoch 15/100
202/202 - 1s - loss: 0.0035 - 977ms/epoch - 5ms/step
Epoch 16/100
202/202 